
# **Evaluación Parcial 1 — CRISP‑DM (Fases 1–3)**  
**Caso:** *Gaming Trends 2024*  
**Asignatura:** MLY0100 — Machine Learning  
**Formato:** Jupyter Notebook (Python)

> Este notebook cubre **las tres primeras fases de CRISP‑DM** (Comprensión del Negocio, Comprensión de los Datos, Preparación de los Datos) con el dataset *Gaming-Trends-2024.csv*.



## 1) **Comprensión del Negocio (Business Understanding)**
**Objetivo general:** Identificar **palancas de crecimiento e ingresos** para títulos/juegos a lo largo del tiempo y por plataforma/género, para orientar decisiones de adquisición, engagement y monetización.

**Preguntas guía e hipótesis operativas:**
- **H1 (Adquisición):** A mayor **exposición orgánica** (menciones en redes) y **campañas con influencers**, mayor **nuevos registros** y **DAU**.
- **H2 (Engagement):** Mayor **duración de sesión** se asocia a mayores **DAU** e **ingresos in‑game**.
- **H3 (Monetización):** **DAU** y **audiencia en streaming** predicen **Revenue**; el efecto varía por **plataforma** y **género**.
- **H4 (Plataforma):** **Móvil** convierte mejor en **nuevos registros**; **PC/Consola** muestran **ARPU** más alto.

**Tareas de ML a futuro (definición de *targets*):**
- **Regresión:** *Revenue* como variable objetivo continua.
- **Clasificación:** *HighRevenue* (binaria) creada a partir de un umbral (p. ej., mediana de *Revenue*).

> Estas definiciones podrán ajustarse tras la comprensión y perfilado del dataset.



## 2) **Comprensión de los Datos (Data Understanding)**
**Acciones:** Carga del dataset, inspección de estructura, tipos de datos, conteos, estadísticos descriptivos, distribuciones, correlaciones y revisión de **missing values** / **outliers**.


In [10]:

import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, classification_report, roc_auc_score, roc_curve

# Configuración general
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)
plt.rcParams.update({"figure.figsize": (8, 5)})


ModuleNotFoundError: No module named 'sklearn'

In [ ]:

# Carga del dataset (asegúrate de tener el CSV en la misma carpeta que este notebook)
csv_path = "Gaming-Trends-2024.csv"
assert os.path.exists(csv_path), "No se encontró el archivo Gaming-Trends-2024.csv en el directorio actual."

df = pd.read_csv(csv_path)
print("Shape:", df.shape)
print("Columnas:", list(df.columns))
df.head()


In [ ]:

# Tipos y valores faltantes
display(df.dtypes)
na_counts = df.isna().sum().sort_values(ascending=False)
display(na_counts.to_frame(name="missing"))

# Duplicados
print("Duplicados:", df.duplicated().sum())



**Diccionario de variables (esperado en el dataset):**
- `Date`: fecha (temporal).
- `Platform`: plataforma (PC, Consola, Móvil, VR).
- `Daily Active Users (DAU)`: usuarios activos diarios.
- `New Registrations`: nuevos registros.
- `Session Duration (min)`: minutos por sesión (promedio).
- `In-game Purchases ($)`: ingresos por compras dentro del juego.
- `Social Media Mentions`: menciones orgánicas en redes.
- `Stream Viewership`: audiencia en streaming.
- `Revenue ($)`: ingresos totales.
- `Top Genre`: género principal del juego.
- `Influencer Endorsements`: 0/1 si hubo endorsos de influencers.

> Si algún nombre difiere, el código intentará resolverlo automáticamente.


In [ ]:

def find_col(df, key_candidates):
    # Try to find a column in df whose lowercased name contains ALL tokens in any candidate string.
    # Returns the best match or raises ValueError if not found.
    cols = list(df.columns)
    lower = [c.lower() for c in cols]
    scores = []
    for cand in key_candidates:
        tokens = [t.strip() for t in cand.lower().replace("(", " ").replace(")", " ").replace("$"," ").split() if t.strip()]
        for i, lc in enumerate(lower):
            if all(tok in lc for tok in tokens):
                scores.append((i, cols[i], len(tokens)))
    if not scores:
        raise ValueError(f"No se encontró ninguna columna para {key_candidates}")
    # prefer the match with more tokens (more specific)
    scores.sort(key=lambda x: (-x[2], x[0]))
    return scores[0][1]


In [ ]:

# Normalizamos/parseamos la fecha
try:
    date_col = find_col(df, ["date", "fecha"])
except Exception as e:
    raise

df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
assert df[date_col].notna().any(), "La columna de fecha no pudo parsearse. Revisa el formato."

# Extra: variable de tiempo numérica (días desde el mínimo)
t0 = df[date_col].min()
df["t_days"] = (df[date_col] - t0).dt.days

# Identificación de columnas clave (robusta a variaciones de nombre)
revenue_col = find_col(df, ["revenue", "revenue ($)", "ingresos"])
dau_col = find_col(df, ["daily active users", "dau"])
newreg_col = find_col(df, ["new registrations", "nuevos registros"])
sess_col = find_col(df, ["session duration", "session duration (min)", "duración de sesión"])
ingame_col = find_col(df, ["in-game purchases", "in game purchases", "compras in-game"])
social_col = find_col(df, ["social media mentions", "menciones"])
stream_col = find_col(df, ["stream viewership", "stream"])
plat_col = find_col(df, ["platform", "plataforma"])
genre_col = find_col(df, ["top genre", "genre", "género"])
infl_col = find_col(df, ["influencer endorsements", "influencer"])

key_cols = [revenue_col, dau_col, newreg_col, sess_col, ingame_col, social_col, stream_col, plat_col, genre_col, infl_col, "t_days", date_col]
print("Columnas mapeadas:", key_cols)
df[key_cols].head()



### Estadísticos descriptivos (tendencia central y dispersión)
Incluye: media, mediana, moda, varianza, desviación estándar, rango, coeficiente de variación (CV), IQR, mínimos/máximos, suma y conteos.


In [ ]:

def descriptive_table(d):
    num = d.select_dtypes(include=[np.number]).copy()
    summary = pd.DataFrame(index=num.columns)
    summary["count"] = num.count()
    summary["sum"] = num.sum()
    summary["mean"] = num.mean()
    summary["median"] = num.median()
    summary["mode"] = num.mode().iloc[0] if not num.mode().empty else np.nan
    summary["var"] = num.var(ddof=1)
    summary["std"] = num.std(ddof=1)
    summary["min"] = num.min()
    summary["q1"] = num.quantile(0.25)
    summary["q3"] = num.quantile(0.75)
    summary["iqr"] = summary["q3"] - summary["q1"]
    summary["max"] = num.max()
    summary["range"] = summary["max"] - summary["min"]
    summary["cv"] = summary["std"] / summary["mean"]
    return summary

desc = descriptive_table(df)
display(desc.round(3))



### Visualizaciones clave
- **Histogramas** para entender la forma y posibles sesgos.
- **Boxplots** para rangos e **outliers**.
- **Correlaciones** para detectar relaciones entre variables numéricas.


In [ ]:

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# HISTOGRAMS
for col in [c for c in numeric_cols if c not in ["t_days"]][:6]:
    plt.figure()
    df[col].hist(bins=30)
    plt.title(f"Histograma: {col}")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.show()

# BOXPLOTS
for col in [c for c in numeric_cols if c not in ["t_days"]][:6]:
    plt.figure()
    plt.boxplot(df[col].dropna(), vert=True, labels=[col])
    plt.title(f"Boxplot: {col}")
    plt.ylabel(col)
    plt.show()

# CORRELATION HEATMAP (matplotlib only)
corr = df.select_dtypes(include=[np.number]).corr()
plt.figure(figsize=(8,6))
plt.imshow(corr, aspect="auto")
plt.colorbar()
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.title("Matriz de correlación")
plt.tight_layout()
plt.show()



### Calidad de datos: **Missing values** y **Outliers**
- Se inspeccionan faltantes por columna y se propone imputación según el tipo de dato.
- Se detectan outliers (método **IQR**) y se propone *winsorización* (cap) o *clipping* para estabilizar el efecto en modelos sensibles.


In [ ]:

# Missing values
na = df.isna().sum().sort_values(ascending=False)
ax = na.plot(kind="bar")
ax.set_title("Valores faltantes por columna")
ax.set_xlabel("Columna")
ax.set_ylabel("Cantidad de NA")
plt.tight_layout()
plt.show()

# Outliers: detección IQR por columna numérica
num = df.select_dtypes(include=[np.number])
outlier_report = []
for col in num.columns:
    q1, q3 = num[col].quantile(0.25), num[col].quantile(0.75)
    iqr = q3 - q1
    low, high = q1 - 1.5*iqr, q3 + 1.5*iqr
    ol_count = ((num[col] < low) | (num[col] > high)).sum()
    outlier_report.append((col, int(ol_count)))
outlier_df = pd.DataFrame(outlier_report, columns=["col", "outliers"]).sort_values("outliers", ascending=False)
display(outlier_df)



## 3) **Preparación de los Datos (Data Preparation)**
**Objetivos:** imputación de faltantes, tratamiento de outliers, creación de variables y escalamiento.

- **Imputación**: numéricas → mediana; categóricas → moda.
- **Outliers**: *clipping* por percentiles (p.01–p.99) para robustecer.
- **Features adicionales**: `t_days` (ya creada) y métricas derivadas (*ARPU*, *conversion* si procede).
- **Escalamiento**: estandarización vs. normalización según algoritmo.


In [ ]:

df_prep = df.copy()

# Imputación simple
for col in df_prep.columns:
    if df_prep[col].dtype.kind in "biufc":
        med = df_prep[col].median()
        df_prep[col] = df_prep[col].fillna(med)
    else:
        mode_val = df_prep[col].mode().iloc[0] if not df_prep[col].mode().empty else "Desconocido"
        df_prep[col] = df_prep[col].fillna(mode_val)

# Clipping por percentiles para estabilizar extremos en numéricas
num_cols = df_prep.select_dtypes(include=[np.number]).columns
lower = df_prep[num_cols].quantile(0.01)
upper = df_prep[num_cols].quantile(0.99)
df_prep[num_cols] = df_prep[num_cols].clip(lower=lower, upper=upper, axis=1)

# Métricas derivadas (si están disponibles)
try:
    dau_col = find_col(df_prep, ["daily active users", "dau"])
    revenue_col = find_col(df_prep, ["revenue", "ingresos"])
    df_prep["ARPU"] = df_prep[revenue_col] / df_prep[dau_col].replace(0, np.nan)
except Exception:
    df_prep["ARPU"] = np.nan

# Variables categóricas → one-hot encoding
cat_cols = df_prep.select_dtypes(include=["object", "category"]).columns.tolist()
# excluir columna de fecha en texto si existiera
try:
    maybe_date = find_col(df_prep, ["date","fecha"])
    cat_cols = [c for c in cat_cols if c != maybe_date]
except Exception:
    pass

df_model = pd.get_dummies(df_prep, columns=cat_cols, drop_first=True)

print("Shape df_model:", df_model.shape)
df_model.head()



### Demostración: **Regresión** con variable de tiempo (`t_days`) como predictora
**Objetivo:** predecir `Revenue` (target continuo) con un modelo lineal simple/múltiple.  
> *Nota:* Esto es un **primer acercamiento exploratorio**; no busca optimización final del desempeño.


In [ ]:

# Definimos target de REGRESIÓN
revenue_col = find_col(df_model, ["revenue", "ingresos"])

# Features básicas: tiempo + señales de escala
X = df_model[["t_days"]].copy()
y = df_model[revenue_col].copy()

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarización opcional
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

reg = LinearRegression()
reg.fit(X_train_s, y_train)

y_pred = reg.predict(X_test_s)
print("R2:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))

# Visual: y_test vs y_pred
plt.figure()
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel("Real Revenue")
plt.ylabel("Predicho Revenue")
plt.title("Regresión: Real vs Predicho")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()])
plt.show()



### Demostración: **Clasificación** (definición de *target* y baseline)
Se construye `HighRevenue` = 1 si `Revenue` ≥ mediana; 0 en caso contrario.  
Se entrena un modelo logístico simple con `t_days` como señal temporal básica (se puede ampliar con más *features*).


In [ ]:

revenue_col = find_col(df_model, ["revenue", "ingresos"])
threshold = df_model[revenue_col].median()
df_model["HighRevenue"] = (df_model[revenue_col] >= threshold).astype(int)

X = df_model[["t_days"]].copy()
y = df_model["HighRevenue"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_s, y_train)
p = clf.predict_proba(X_test_s)[:,1]
pred = (p >= 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, pred))
try:
    print("ROC AUC:", roc_auc_score(y_test, p))
except Exception as e:
    print("ROC AUC no disponible:", e)

print("\nReporte de clasificación:\n", classification_report(y_test, pred))



### Normalización vs Estandarización (criterios)
- **Estandarización** (`StandardScaler`): útil para modelos lineales/logísticos, SVM, PCA; datos aproximadamente gaussianos.
- **Normalización** (`MinMaxScaler`): útil para redes neuronales o cuando los rangos deben acotarse (0–1).

> La elección depende de la **distribución** y del **algoritmo** objetivo.



## 4) **Documentación y marcadores de la rúbrica**
- Uso de **CRISP‑DM** (secciones 1–3) ✓  
- Identificación de *targets*: **Regresión** (`Revenue`) y **Clasificación** (`HighRevenue`) ✓  
- Uso de librerías: **numpy, pandas, matplotlib, scikit‑learn** ✓  
- Limpieza/preparación: imputación, outliers, *encoding*, *scaling* ✓  
- Estadísticos descriptivos y visualizaciones (histogramas, boxplots, correlaciones) ✓  
- Justificación mediante **markdown** en cada sección ✓

> Este notebook deja la base lista para avanzar a **Modeling** y **Evaluation** conforme a CRISP‑DM.
